In [1]:
#With LCEL you can easily add custom routing logic to your chain to dynamically determine the chain logic based on user input.
#All you need to do is define a function that given an input returns a Runnable.

In [2]:
%pip install --upgrade --quiet  langchain-core langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda , RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [8]:
OPENAI_API_KEY = ""

In [9]:
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)
prompt_templates = [physics_template,math_template]
prompt_embeddings = embedding.embed_documents(prompt_templates)

In [13]:
def prompt_router(input):
    query_embedding =  embedding.embed_query(input['query'])
    similarity = cosine_similarity([query_embedding],prompt_embeddings)
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

In [14]:
chain = (
    {"query":RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI(openai_api_key = OPENAI_API_KEY)
    | StrOutputParser()
)

In [15]:
chain.invoke("What is black Hole")

Using PHYSICS


"A black hole is a region in space where gravity is so strong that nothing, not even light, can escape its pull. This happens when a massive star collapses in on itself, creating a singularity with infinite density at its core. The boundary surrounding the singularity is called the event horizon, and once something crosses this boundary, it is trapped by the black hole's gravity. Black holes come in different sizes, with the supermassive ones found at the centers of galaxies being the largest and most massive."

In [16]:
chain.invoke("Diff b/w sin and cos")

Using MATH


'The main difference between sine (sin) and cosine (cos) functions is their phase shift. \n\nThe sine function is defined as the ratio of the length of the side opposite an acute angle in a right triangle to the length of the hypotenuse. It is also the y-coordinate of a point on the unit circle that is a certain angle counterclockwise from the positive x-axis. The sine function has a phase shift of 90 degrees or π/2 radians, meaning that the sine function starts at a maximum value of 1 when the angle is 0 degrees or 0 radians.\n\nThe cosine function, on the other hand, is defined as the ratio of the length of the side adjacent to an acute angle in a right triangle to the length of the hypotenuse. It is also the x-coordinate of a point on the unit circle that is a certain angle counterclockwise from the positive x-axis. The cosine function has no phase shift, meaning that it starts at a maximum value of 1 when the angle is 0 degrees or 0 radians.\n\nIn summary, the main difference betwe